**4. Widefield Batch Comparison Plotting** 

Written by Jessica Kline

This code is the fourth step in analyzing widefield blinking videos and assumes that "1. Find QDs", "2. Gaussian CPA Fittings" and "3. Widefield Post-CPA Individual Batch Processing" have already been run on all of the necessary data. It reads in the different statistic text files for each batch and plots comparisons of the different batches across a range of statistics.

Statistics
*   Distribution of CPA Levels
*   Expected OFF Event Dwell Time Distribution
*   Expected ON Event Dwell Time Distribution
*   Weighted ON/OFF Ratio Distribution
*   % time ON distribtuion
*   % time OFF distribution
*   Distribution of CPA assigned intensity levels
*   Distribution of Camera measured intensities for all QDs
*   QD Dying test
*   Time to First Dark Event
*   Non-blinking fraction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.ticker import MultipleLocator
import math
import os
from os import listdir
from os.path import isfile, join
import seaborn as sns

import logging

Mounted at /content/drive


In [ ]:
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 22
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['xtick.major.pad']='10'
plt.rcParams['ytick.major.pad']='4'

logging.getLogger('matplotlib.font_manager').disabled = True

In [ ]:
primary_color_scheme = ['#e50000', '#0043C5', '#FFAE03']
shades_of_red = ['#590000','#e50000', '#ff976a']
shades_of_blue = ['#001b4f','#002f8a','#0043c5', '#4d7bd6', '#99b4e8']

# dark_shades_of_blue = ['#001b4f','#002f8a','#0043c5']
# light_shades_of_blue = ['#0043c5','#4d7bd6', '#99b4e8']

In [ ]:
#@title tick_settings
def tick_settings(x_minor = 1, y_minor = 1):
  plt.gca().xaxis.set_minor_locator(MultipleLocator(x_minor))
  plt.gca().yaxis.set_minor_locator(MultipleLocator(y_minor))
  plt.gca().tick_params(which='minor', bottom=True, top=True, left=True, right=True)
  plt.gca().tick_params(bottom=True, top=True, left=True, right=True)
  plt.tick_params(axis = 'both', direction = 'in', length = 6, width = 1.5)
  plt.gca().tick_params(which='minor', direction = "in", length = 4, width = 0.75)

  for axis in ['top','bottom','left','right']:
    plt.gca().spines[axis].set_linewidth(1.5)

This code expects the following file tree:
1. All_Groups = directory_path/Large Data Folder/analyzed_data/ for multiple different batches, each of which contain the results of "3. Widefield Post-CPA Individual Batch Processing"
2. save_location - a pre-existing folder that you want the comparison images and text files saved to

This code creates:
1. save_location/save_folder_name - a folder within you pre-existing save path to save all the comparison images and text files to




In [ ]:
All_Groups = ['//InsertYourPathHere/analyzed_data//','//InsertYourPathHere/analyzed_data//']
All_Names = ['batch name', 'batch name']
num_groups = len(All_Groups)

bin_size_ms = 50        #time bins size in ms
total_frames = 8000     #total frames in the videos

color_scheme = primary_color_scheme   #plotting colors

save_location = "//InsertYourSavePathHere//"
save_folder_name = "//InsertYourFolderNameHere//"

if os.path.exists(save_location+save_folder_name) == False:
  os.mkdir(save_location+save_folder_name)

In [ ]:
#@title batch statistics

#                           g1  |  s1   |   g2    | s2
# sample size             |     |
# ON%                     |     |                       
# weighted on/off         |     |
# avg levels              |     |
# <t on>
# <t off>
# non-blinking fration    |     |                         #defined as 95% on

batch_stats = np.zeros([7,num_groups*2])

In [ ]:
#@title Num CPA Levels
width_ = 0.75/num_groups
max_bin = 6
bin_starts = np.repeat(np.arange(0.625,1.37,width_),max_bin)
bin_starts = bin_starts.reshape([num_groups,-1])

total_lst = []

# total_lst.extend(np.arange(1,max_bin))

for n in range(len(bin_starts[0,:])):
  for k in range(num_groups):
    bin_starts[k,n] += n


for iter in range(num_groups):
  Group_states = []
  for n in range(len(All_Groups[iter][:])):
    grp1_txt = open(All_Groups[iter][n]+"Statistics.csv", "r")
    grp1_txt = grp1_txt.read()
    grp1_txt = grp1_txt.split()
    grp1_txt = grp1_txt[5:]
    temp = []
    for x in grp1_txt:
      j = x.split(",")
      for num in j:
        temp.append(num)
    temp = np.array([float(x) if x != '' else math.nan for x in temp])
    temp = temp.reshape([int(len(temp)/5),5])
    Group_states = np.append(Group_states, temp[:,1])


  batch_stats[0,iter*2] = len(Group_states)
  h = np.histogram(Group_states, bins = np.arange(1,max_bin+2))

  batch_stats[3,iter*2] = np.mean(Group_states)
  batch_stats[3,(iter*2)+1] = np.std(Group_states)

  group_norm = h[0]/np.sum(h[0])

  total_lst.extend(group_norm)

  plt.bar(bin_starts[iter,:], group_norm, width = width_, align = 'edge', edgecolor = 'k', label = All_Names[iter], color = color_scheme[iter])

tick_settings(x_minor = 0.5, y_minor = 0.05)
plt.xlabel('# of CPA Levels')
plt.ylabel('fraction of dots')
plt.legend()
plt.xlim([0.5, 4.5])

ff = plt.gcf()
ff.set_figwidth(6)
ff.set_figheight(4)

plt.savefig(save_location+save_folder_name+"NumberOfLevels.jpg", bbox_inches = 'tight')

total_lst = np.reshape(total_lst, [num_groups, -1])
total_lst = pd.DataFrame(total_lst.T)

total_lst.columns = All_Names
total_lst.index = np.arange(1,max_bin+1)

total_lst.to_csv(save_location+save_folder_name+"NumberOfLevels.csv")


In [ ]:
#@title weighted on/off ratio
max_ratio = []
total_lst = []

for iter in range(num_groups):
  Group_onoff = []
  Group_on = []
  Group_off = []
  for n in range(len(All_Groups[iter][:])):
    grp1_txt = open(All_Groups[iter][n]+"Statistics.csv", "r")
    grp1_txt = grp1_txt.read()
    grp1_txt = grp1_txt.split()
    grp1_txt = grp1_txt[5:]
    temp = []
    for x in grp1_txt:
      j = x.split(",")
      for num in j:
        temp.append(num)
    temp = np.array([float(x) if x != '' else math.nan for x in temp])
    temp = temp.reshape([int(len(temp)/5),5])
    Group_onoff = np.append(Group_onoff, temp[:,4])
    Group_on = np.append(Group_on, temp[:,2])
    Group_off = np.append(Group_off, temp[:,3])

  batch_stats[2,iter*2] = np.nanmean(Group_onoff)
  batch_stats[2,(iter*2)+1] = np.nanstd(Group_onoff)

  max_ratio = np.append(max_ratio,np.nanmax(Group_onoff))

  h = np.histogram(Group_onoff, bins = np.arange(0,20,0.2))
  group_norm = h[0]/np.sum(h[0])

  total_lst.extend(group_norm)

  plt.bar(h[1][:-1], group_norm, width = 0.2, label = All_Names[iter], color = color_scheme[iter], alpha = 0.7)

plt.legend()
plt.xlim(0,10)
tick_settings(x_minor = 0.5, y_minor = 0.01)
plt.xlabel('weighted on/off ratio')
plt.ylabel('fraction of dots')
plt.legend()

ff = plt.gcf()
ff.set_figwidth(8)
ff.set_figheight(4)

plt.savefig(save_location+save_folder_name+"WeightedOnOff.jpg", bbox_inches = 'tight')

total_lst = np.reshape(total_lst, [num_groups, -1])
total_lst = pd.DataFrame(total_lst.T)

total_lst.columns = All_Names
total_lst.index = h[1][:-1]
total_lst.to_csv(save_location+save_folder_name+"WeightedOnOff.csv")

In [ ]:
#@title time on
max_ratio = []
total_lst = []
for iter in range(num_groups):
  Group_on = []
  for n in range(len(All_Groups[iter][:])):
    grp1_txt = open(All_Groups[iter][n]+"Statistics.csv", "r")
    grp1_txt = grp1_txt.read()
    grp1_txt = grp1_txt.split()
    grp1_txt = grp1_txt[5:]
    temp = []
    for x in grp1_txt:
      j = x.split(",")
      for num in j:
        temp.append(num)
    temp = np.array([float(x) if x != '' else math.nan for x in temp])
    temp = temp.reshape([int(len(temp)/5),5])
    Group_on = np.append(Group_on, temp[:,2])

  batch_stats[4,iter*2] = np.nanmean(Group_on)
  batch_stats[4,(iter*2)+1] = np.nanstd(Group_on)

  max_ratio = np.append(max_ratio,np.nanmax(Group_on))

  h = np.histogram(Group_on, bins = np.arange(0,20,0.02))
  group_norm = h[0]/np.sum(h[0])
  total_lst.extend(group_norm)

  plt.bar(h[1][:-1], group_norm, width = 0.02, label = All_Names[iter], color = color_scheme[iter], alpha = 0.7)

plt.legend()
plt.xlim(0,4)
tick_settings(x_minor = 0.1, y_minor = 0.01)
plt.xlabel('event <t on> (s)')
plt.ylabel('fraction of dots')
plt.legend()

ff = plt.gcf()
ff.set_figwidth(8)
ff.set_figheight(4)

plt.savefig(save_location+save_folder_name+"TimeOn.jpg", bbox_inches = 'tight')

total_lst = np.reshape(total_lst, [num_groups, -1])
total_lst = pd.DataFrame(total_lst.T)

total_lst.columns = All_Names
total_lst.index = h[1][:-1]
total_lst.to_csv(save_location+save_folder_name+"TimeOn.csv")

In [ ]:
#@title time off
max_ratio = []
total_lst = []
for iter in range(num_groups):
  Group_off = []
  for n in range(len(All_Groups[iter][:])):
    grp1_txt = open(All_Groups[iter][n]+"Statistics.csv", "r")
    grp1_txt = grp1_txt.read()
    grp1_txt = grp1_txt.split()
    grp1_txt = grp1_txt[5:]
    temp = []
    for x in grp1_txt:
      j = x.split(",")
      for num in j:
        temp.append(num)
    temp = np.array([float(x) if x != '' else math.nan for x in temp])
    temp = temp.reshape([int(len(temp)/5),5])
    Group_off = np.append(Group_off, temp[:,3])

  batch_stats[5,iter*2] = np.nanmean(Group_off)
  batch_stats[5,(iter*2)+1] = np.nanstd(Group_off)

  max_ratio = np.append(max_ratio,np.nanmax(Group_off))

  h = np.histogram(Group_off, bins = np.arange(0,20,0.02))
  group_norm = h[0]/np.sum(h[0])

  total_lst.extend(group_norm)

  plt.bar(h[1][:-1], group_norm, width = 0.02, label = All_Names[iter], color = color_scheme[iter], alpha = 0.7)

plt.legend()
plt.xlim(0,2)
tick_settings(x_minor = 0.1, y_minor = 0.01)
plt.xlabel('event <t off> (s)')
plt.ylabel('fraction of dots')
plt.legend()

ff = plt.gcf()
ff.set_figwidth(8)
ff.set_figheight(4)

plt.savefig(save_location+save_folder_name+"TimeOff.jpg", bbox_inches = 'tight')

total_lst = np.reshape(total_lst, [num_groups, -1])
total_lst = pd.DataFrame(total_lst.T)

total_lst.columns = All_Names
total_lst.index = h[1][:-1]
total_lst.to_csv(save_location+save_folder_name+"TimeOff.csv")

In [ ]:
#@title %time spent in dark
width_ = 4/num_groups
max_bin = 21
bin_starts = np.repeat(np.arange(-2,2,width_),max_bin)
bin_starts = bin_starts.reshape([num_groups,-1])

total_lst = []

for n in range(len(bin_starts[0,:])):
  for k in range(num_groups):
    bin_starts[k,n] += n*5

for iter in range(num_groups):
  Group_dark = []
  for n in range(len(All_Groups[iter][:])):
    grp1_txt = open(All_Groups[iter][n]+"DarkFraction.csv", "r")
    grp1_txt = grp1_txt.read()
    grp1_txt = grp1_txt.split()
    grp1_txt = grp1_txt[3:]
    temp = []
    for x in grp1_txt:
      j = x.split(",")
      for num in j:
        temp.append(num)
    temp = np.array([float(x) for x in temp])
    temp = temp.reshape([int(len(temp)/3),3])
    if n == 0:
      Group_dark.append(temp[:,1])
      Group_dark.append(temp[:,2])
    else:
      Group_dark[1] += temp[:,2]
  Group_dark[1] /= len(All_Groups[iter][:])

  total_lst.extend(Group_dark[1])

  plt.bar(bin_starts[iter,:], Group_dark[1], width = width_, align = 'edge', edgecolor = 'k', label = All_Names[iter], color = color_scheme[iter])
tick_settings(x_minor = 5, y_minor = 0.05)
plt.xticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
plt.xlim([-2,100])
plt.xlabel('% OFF')
plt.ylabel('fraction of dots')
plt.legend()

ff = plt.gcf()
ff.set_figwidth(8)
ff.set_figheight(4)

plt.savefig(save_location+save_folder_name+"DarkFraction.jpg", bbox_inches = 'tight')

total_lst = np.reshape(total_lst, [num_groups, -1])
total_lst = pd.DataFrame(total_lst.T)

total_lst.columns = All_Names
total_lst.index = np.arange(0,105,5)#h[1][:-1]
total_lst.to_csv(save_location+save_folder_name+"DarkFraction.csv")

In [ ]:
#@title %time spent in light

width_ = 4/num_groups
max_bin = 21
bin_starts = np.repeat(np.arange(3,7,width_),max_bin)
bin_starts = bin_starts.reshape([num_groups,-1])

total_lst = []

for n in range(len(bin_starts[0,:])):
  for k in range(num_groups):
    bin_starts[k,n] += n*5

for iter in range(num_groups):
  Group_light = []
  for n in range(len(All_Groups[iter][:])):
    grp1_txt = open(All_Groups[iter][n]+"LightFraction.csv", "r")
    grp1_txt = grp1_txt.read()
    grp1_txt = grp1_txt.split()
    grp1_txt = grp1_txt[3:]
    temp = []
    for x in grp1_txt:
      j = x.split(",")
      for num in j:
        temp.append(num)
    temp = np.array([float(x) for x in temp])
    temp = temp.reshape([int(len(temp)/3),3])
    if n == 0:
      Group_light.append(temp[:,1])
      Group_light.append(temp[:,2])
    else:
      Group_light[1] += temp[:,2]
  Group_light[1] /= len(All_Groups[iter][:])

  total_lst.extend(Group_light[1])


  batch_stats[6,iter*2] = Group_light[1][-2]

  plt.bar(bin_starts[iter,:], Group_light[1], width = width_, align = 'edge', edgecolor = 'k', label = All_Names[iter], color = color_scheme[iter])
tick_settings(x_minor = 5, y_minor = 0.05)
plt.xticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
plt.xlim([0,102])
plt.xlabel('% ON')
plt.ylabel('fraction of dots')
plt.legend()

ff = plt.gcf()
ff.set_figwidth(8)
ff.set_figheight(4)

plt.savefig(save_location+save_folder_name+"LightFraction.jpg", bbox_inches = 'tight')

total_lst = np.reshape(total_lst, [num_groups, -1])
total_lst = pd.DataFrame(total_lst.T)

total_lst.columns = All_Names
total_lst.index = np.arange(0,105,5)#h[1][:-1]
total_lst.to_csv(save_location+save_folder_name+"LightFraction.csv")

In [ ]:
#@title On%
for iter in range(num_groups):
  Group_light = []
  tot_num = 0
  for n in range(len(All_Groups[iter][:])):
    grp1_txt = open(All_Groups[iter][n]+"OnPercent.csv", "r")
    grp1_txt = grp1_txt.read()
    grp1_txt = grp1_txt.split()
    grp1_txt = grp1_txt[2:]
    temp = []
    for x in grp1_txt:
      j = x.split(",")
      for num in j:
          temp.append(num)
    temp = np.array([float(x) for x in temp])

    batch_stats[1,iter*2] += (temp[1]*temp[3])
    batch_stats[1,(iter*2)+1] += (temp[1]*temp[5])
    tot_num += temp[1]

  batch_stats[1,iter*2] /= tot_num
  batch_stats[1,(iter*2)+1] /= tot_num

In [ ]:
#@title time to first dark event
total_lst = []

for iter in range(num_groups):
  Group_timedark = []
  for n in range(len(All_Groups[iter][:])):
    grp1_txt = open(All_Groups[iter][n]+"TimeToDark.csv", "r")
    grp1_txt = grp1_txt.read()
    grp1_txt = grp1_txt.split()
    grp1_txt = grp1_txt[3:]
    temp = []
    for x in grp1_txt:
      j = x.split(",")
      for num in j:
        temp.append(num)
    temp = np.array([float(x) for x in temp])
    temp = temp.reshape([int(len(temp)/3),3])
    if n == 0:
      Group_timedark.append(temp[:,1])
      Group_timedark.append(temp[:,2])
    else:
      Group_timedark[1] += temp[:,2]
  Group_timedark[1] /= len(All_Groups[iter][:])

  Group_timedark[0]= np.append(Group_timedark[0], 9999)
  Group_timedark[1] = np.append(Group_timedark[1], Group_timedark[1][-1])

  total_lst.extend(Group_timedark[1])

  plt.plot(Group_timedark[0], Group_timedark[1], linewidth = 4, label = All_Names[iter], color = color_scheme[iter])

plt.ylim([0,1])
plt.xlim([0, 250])
plt.xlabel('time to first dark event (s)')
plt.ylabel('on fraction')
tick_settings(x_minor = 10, y_minor = 0.05)
plt.legend()

ff = plt.gcf()
ff.set_figwidth(6)
ff.set_figheight(4)


plt.savefig(save_location+save_folder_name+"TimeToDark.jpg", bbox_inches = 'tight')

total_lst = np.reshape(total_lst, [num_groups, -1])
total_lst = pd.DataFrame(total_lst.T)

total_lst.columns = All_Names
total_lst.index = Group_timedark[0]
total_lst.to_csv(save_location+save_folder_name+"TimeToDark.csv")

In [ ]:
#@title CPA intensity distribution
total_lst = []

for iter in range(num_groups):
  Group_cpaInt = []
  for n in range(len(All_Groups[iter][:])):
    grp1_txt = open(All_Groups[iter][n]+"CPAIntensityDistribution.csv", "r")
    grp1_txt = grp1_txt.read()
    grp1_txt = grp1_txt.split()
    grp1_txt = grp1_txt[3:]
    temp = []
    for x in grp1_txt:
      j = x.split(",")
      for num in j:
        temp.append(num)
    temp = np.array([float(x) for x in temp])
    temp = temp.reshape([int(len(temp)/3),3])
    if n == 0:
      Group_cpaInt.append(temp[:,1])
      Group_cpaInt.append(temp[:,2])
    else:
      Group_cpaInt[1] += temp[:,2]
  Group_cpaInt[1] /= len(All_Groups[iter][:])
  Group_cpaInt[1] /= np.amax(Group_cpaInt[1])

  total_lst.extend(Group_cpaInt[1])

  plt.bar(Group_cpaInt[0], Group_cpaInt[1], width = 0.5, label = All_Names[iter], color = color_scheme[iter], alpha = 0.7)

plt.legend()
plt.xlabel('Assigned CPA Intensity (counts/50 ms)')
plt.ylabel('Norm. Occurrences (au)')
tick_settings(x_minor = 10, y_minor = 0.1)
plt.ylim([0,1])
plt.xlim(100 , 150)

ff = plt.gcf()
ff.set_figwidth(8)
ff.set_figheight(4)

plt.savefig(save_location+save_folder_name+"CPAIntensityDistribution.jpg", bbox_inches = 'tight')

total_lst = np.reshape(total_lst, [num_groups, -1])
total_lst = pd.DataFrame(total_lst.T)

total_lst.columns = All_Names
total_lst.index = Group_cpaInt[0]
total_lst.to_csv(save_location+save_folder_name+"CPAIntensityDistribution.csv")

In [ ]:
#@title camera int dist
total_lst = []

for iter in range(num_groups):
  Group_cameraInt = []
  for n in range(len(All_Groups[iter][:])):
    grp1_txt = open(All_Groups[iter][n]+"CameraIntensityDistribution.csv", "r")
    grp1_txt = grp1_txt.read()
    grp1_txt = grp1_txt.split()
    grp1_txt = grp1_txt[3:]
    temp = []
    for x in grp1_txt:
      j = x.split(",")
      for num in j:
        temp.append(num)
    temp = np.array([float(x) for x in temp])
    temp = temp.reshape([int(len(temp)/3),3])
    if n == 0:
      Group_cameraInt.append(temp[:,1])
      Group_cameraInt.append(temp[:,2])
    else:
      Group_cameraInt[1] += temp[:,2]
  Group_cameraInt[1] /= len(All_Groups[iter][:])
  Group_cameraInt[1] /= np.amax(Group_cameraInt[1])
  
  total_lst.extend(Group_cameraInt[1])

  plt.bar(Group_cameraInt[0], Group_cameraInt[1], width =0.5, label = All_Names[iter], color = color_scheme[iter], alpha = 0.7)

plt.legend()
plt.xlabel('Camera Counts (counts/50 ms)')
plt.ylabel('Norm. Occurrences (au)')
tick_settings(x_minor = 10, y_minor = 0.1)
plt.ylim([0,1])
plt.xlim(100 , 150)

ff = plt.gcf()
ff.set_figwidth(8)
ff.set_figheight(4)

plt.savefig(save_location+save_folder_name+"CameraIntensityDistribution.jpg", bbox_inches = 'tight')


total_lst = np.reshape(total_lst, [num_groups, -1])
total_lst = pd.DataFrame(total_lst.T)

total_lst.columns = All_Names
total_lst.index = Group_cpaInt[0]
total_lst.to_csv(save_location+save_folder_name+"CameraIntensityDistribution.csv")

In [ ]:
#@title dying/drift check

bins_ = np.arange(80, 298.5, 0.5)
time_check = np.arange(0,total_frames,int(total_frames/10))

for iter in range(num_groups):
  Group_death = []
  for n in range(len(All_Groups[iter][:])):
    grp1_txt = open(All_Groups[iter][n]+"DyingCheck.csv", "r")
    grp1_txt = grp1_txt.read()
    grp1_txt = grp1_txt.split()
    grp1_txt = grp1_txt[3:]
    temp = []
    for x in grp1_txt:
      j = x.split(",")
      for num in j:
        temp.append(num)
    temp = np.array([float(x) for x in temp])
    temp = temp.reshape([-1,11])
    if n == 0:
      Group_death = temp
    else:
      Group_death += temp
  Group_death /= len(All_Groups[iter][:])

  for cnt in range (1,11):
    plt.subplot(10,1, cnt)
    plt.bar(bins_, Group_death[:,cnt], width = 0.5,label = All_Names[iter], color = color_scheme[iter], alpha = 0.7)
    if cnt == 10:
      plt.xlabel(f'Raw Camera Intensity (counts/{bin_size_ms} ms)')
    else:
      plt.xticks([])
    if cnt == 6:
      plt.ylabel('Norm. Occurrences (au)')
      plt.legend(loc=(1.5, 1.5))
    plt.text(131, 0.5, "{:.1f}".format(time_check[cnt-1]*bin_size_ms/60000)+ ' mins') 
    tick_settings(x_minor = 5, y_minor = 0.2)
    plt.ylim([0,1])
    plt.xlim([95, 130])

ff = plt.gcf()
ff.set_figwidth(5)
ff.set_figheight(15)

plt.savefig(save_location+save_folder_name+"DyingCheck.jpg", bbox_inches = 'tight')

In [ ]:
#@title non_blinking fraction
for n in range(0,num_groups*2,2):
  plt.bar([n/2],[batch_stats[6,n]],  width = 1, align = 'edge', edgecolor = 'k', label = All_Names[int(n/2)], color = color_scheme[int(n/2)])

plt.xticks([])
plt.ylabel('Non-Bliking Fraction')
plt.legend(loc = (1.01,0))
#plt.xticklabels(All_Names)
tick_settings(x_minor = 10, y_minor = 0.05)

plt.savefig(save_location+save_folder_name+"NonBlinkingFraction.jpg", bbox_inches = 'tight')

In [ ]:
batch_stats = pd.DataFrame(batch_stats.T)
names = np.repeat(All_Names,2)

batch_stats.columns = ['num dots', 'ON%', 'weighted on/off', 'avg CPA levels', '<t> on (s)', '<t> off (s)', 'non-blinking fraction']
batch_stats.index = names

print(batch_stats)
batch_stats.to_csv(save_location+save_folder_name+"Batch Statistics.csv")